In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data-preprocessing/csv/ohlcv_with_future_tan.csv')

In [3]:
close_c_30min = df['close_c_30min'].copy()
df = df.drop(columns=['close_c_30min'], axis=1)

In [4]:
features_np = df.values
answer_np = close_c_30min.values

In [5]:
X, y = list(), list()

for i in range(60, features_np.shape[0]):
    X.append(features_np[i-60:i])
    y.append(answer_np[i])
    
X, y = np.array(X), np.array(y)

In [6]:
X.shape, y.shape

((143877, 60, 14), (143877,))

In [7]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

n_layer_lstm = 3
lstm_unit = 8
n_layer_dense = 3
dense_unit = 16
dense_dropout = 0.3
lstm_dropout = 0.2
act_fn = 'relu'

def get_model(n_layer_lstm, lstm_unit, n_layer_dense, dense_unit, dense_dropout, lstm_dropout, act_fn, lr):
    inp = Input(shape=(60,14))
    for i in range(n_layer_lstm):
        if i == n_layer_lstm-1 and i != 0:
            x = LSTM(lstm_unit, dropout=lstm_dropout)(x)
        elif i == n_layer_lstm-1 and i == 0:
            x = LSTM(lstm_unit, dropout=lstm_dropout)(inp)
        elif i == 0:
            x = LSTM(lstm_unit, return_sequences=True, dropout=lstm_dropout)(inp)
        else:
            x = LSTM(lstm_unit, return_sequences=True, dropout=lstm_dropout)(x)
    for i in range(n_layer_dense):
        x = Dense(dense_unit)(x)
        x = BatchNormalization()(x)
        x = Activation(act_fn)(x)
        x = Dropout(dense_dropout)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, out)
    model.compile(optimizers=Adam(lr), loss='binary_crossentropy', metrics=["accuracy"])
    
    return model

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', mode="min", patience=10)

K.clear_session()
for n_layer_lstm in range(1,4):
    for lstm_unit in [32, 16, 64]:
        for n_layer_dense in [0, 1, 2, 3]:
            print('>'*20,'{} LSTM layers with {} units and {} layer of DNN'.format(n_layer_lstm, lstm_unit, n_layer_dense), '<'*20)
            model = get_model(n_layer_lstm = n_layer_lstm,
                              lstm_unit = lstm_unit,
                              n_layer_dense = n_layer_dense,
                              dense_unit = lstm_unit,
                              dense_dropout = 0.,
                              lstm_dropout = 0.,
                              act_fn = 'relu',
                              lr=1e-3)
            model.summary()
            checkpoint = ModelCheckpoint(filepath='best_params_{}_layer_{}_units_{}_dense'.format(n_layer_lstm, lstm_unit, n_layer_dense), monitor="val_accuracy",verbose = 1, save_best_only=True )
            model.fit(X, y, validation_split=0.2, epochs=100, batch_size=256, callbacks=[checkpoint, early_stopping], verbose=1)

>>>>>>>>>>>>>>>>>>>> 1 LSTM layers with 32 units and 0 layer of DNN <<<<<<<<<<<<<<<<<<<<
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 60, 14)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6016      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 6,049
Trainable params: 6,049
Non-trainable params: 0
_________________________________________________________________
Train on 115101 samples, validate on 28776 samples
Epoch 1/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6887 - accuracy: 0.5347
Epoch 00001: val_accuracy improved from -inf to 0.55362, saving model to best_params_1_layer_32_units_0_dense
Instructions for updating:
If using 

114944/115101 [============================>.] - ETA: 0s - loss: 0.6551 - accuracy: 0.6076
Epoch 00023: val_accuracy did not improve from 0.56019
115101/115101 [==============================] - 46s 401us/sample - loss: 0.6551 - accuracy: 0.6075 - val_loss: 0.7121 - val_accuracy: 0.5372
Epoch 24/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6527 - accuracy: 0.6112
Epoch 00024: val_accuracy did not improve from 0.56019
115101/115101 [==============================] - 48s 414us/sample - loss: 0.6527 - accuracy: 0.6112 - val_loss: 0.7226 - val_accuracy: 0.5412
Epoch 25/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6501 - accuracy: 0.6149
Epoch 00025: val_accuracy did not improve from 0.56019
115101/115101 [==============================] - 48s 417us/sample - loss: 0.6501 - accuracy: 0.6149 - val_loss: 0.7426 - val_accuracy: 0.5309
Epoch 26/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6476 - accuracy: 0.6188
Epoch 

114944/115101 [============================>.] - ETA: 0s - loss: 0.6864 - accuracy: 0.5507
Epoch 00001: val_accuracy improved from -inf to 0.50142, saving model to best_params_1_layer_32_units_2_dense
INFO:tensorflow:Assets written to: best_params_1_layer_32_units_2_dense\assets
115101/115101 [==============================] - 60s 520us/sample - loss: 0.6864 - accuracy: 0.5506 - val_loss: 0.6988 - val_accuracy: 0.5014
Epoch 2/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6824 - accuracy: 0.5606
Epoch 00002: val_accuracy improved from 0.50142 to 0.51908, saving model to best_params_1_layer_32_units_2_dense
INFO:tensorflow:Assets written to: best_params_1_layer_32_units_2_dense\assets
115101/115101 [==============================] - 57s 493us/sample - loss: 0.6824 - accuracy: 0.5606 - val_loss: 0.6985 - val_accuracy: 0.5191
Epoch 3/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6812 - accuracy: 0.5636
Epoch 00003: val_accuracy improved fro

114944/115101 [============================>.] - ETA: 0s - loss: 0.6722 - accuracy: 0.5812 ETA: 5s - loss: 0.6723 - accuracy - ETA: 5s - los - ETA: 1s - loss: 0.6721 - accuracy: 0.58 - ETA: 1s
Epoch 00007: val_accuracy improved from 0.54076 to 0.54295, saving model to best_params_1_layer_32_units_3_dense
INFO:tensorflow:Assets written to: best_params_1_layer_32_units_3_dense\assets
115101/115101 [==============================] - 59s 511us/sample - loss: 0.6722 - accuracy: 0.5812 - val_loss: 0.7473 - val_accuracy: 0.5430
Epoch 8/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6702 - accuracy: 0.5841
Epoch 00008: val_accuracy did not improve from 0.54295
115101/115101 [==============================] - 45s 395us/sample - loss: 0.6702 - accuracy: 0.5841 - val_loss: 0.7310 - val_accuracy: 0.5269
Epoch 9/100
114944/115101 [============================>.] - ETA: 0s - loss: 0.6678 - accuracy: 0.5896
Epoch 00009: val_accuracy did not improve from 0.54295
115101/115101 [

114944/115101 [============================>.] - ETA: 0s - loss: 0.6803 - accuracy: 0.5642
Epoch 00015: val_accuracy did not improve from 0.55887
115101/115101 [==============================] - 49s 424us/sample - loss: 0.6804 - accuracy: 0.5642 - val_loss: 0.6867 - val_accuracy: 0.5448
>>>>>>>>>>>>>>>>>>>> 1 LSTM layers with 16 units and 1 layer of DNN <<<<<<<<<<<<<<<<<<<<
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 60, 14)]          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 16)                1984      
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
_________________________________________________________________
batch_normalization_6 (Batch (None, 16)                64        
____________